# Bigram Character Language Model (PyTorch)

This notebook teaches you how a **bigram character language model** works: it predicts the next character using only the current character. It is a tiny but complete language model that helps you understand the core ideas behind LLMs?tokenization, next-token prediction, cross-entropy loss, and sampling?without the complexity of attention or deep networks.

## 1) Setup + imports
We will use PyTorch for modeling and the Hugging Face `datasets` library for loading text. We'll also set global configuration values to keep training reproducible and easy to tweak.

In [14]:

import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version (torch):", torch.version.cuda)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


Torch version: 2.5.1+cu121
CUDA available: True
CUDA version (torch): 12.1
GPU: NVIDIA GeForce RTX 3070 Laptop GPU


In [15]:
import math
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset

# ---- Global config ----
SEED = 42  # random seed
BATCH_SIZE = 32  # batch size
BLOCK_SIZE = 64  # context length
EMBED_DIM = None  # set to e.g. 64 to use embedding->linear, or None for pure bigram
LR = 1e-2  # learning rate
MAX_STEPS = 800  # training steps
EVAL_EVERY = 200  # eval interval (steps)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # compute device

torch.manual_seed(SEED)
random.seed(SEED)

print('Device:', DEVICE)

Device: cuda


## 2) Load dataset
We load `wikitext-2-raw-v1` from Hugging Face. This notebook assumes that dataset is available.


In [16]:
ds = load_dataset('wikitext', 'wikitext-2-raw-v1')
dataset_name = 'wikitext-2-raw-v1'
print('Loaded dataset:', dataset_name)

# Optional cleanup: keep ASCII only so the character vocab stays simple

# Most text datasets provide a single train split with a 'text' column.
text = ''
if 'train' in ds:
    text = '\n'.join(ds['train']['text'])
else:
    # fallback if dataset has only one split
    first_split = list(ds.keys())[0]
    text = '\n'.join(ds[first_split]['text'])

# Filter to ASCII to avoid a huge or noisy character set
text = ''.join(ch for ch in text if ch.isascii())
print('Text length:', len(text))

Could not load tiny_shakespeare: Dataset scripts are no longer supported, but found tiny_shakespeare.py
Falling back to shakespeare...
Could not load shakespeare: Dataset 'shakespeare' doesn't exist on the Hub or cannot be accessed.
Falling back to wikitext-2-raw-v1...
Loaded dataset: wikitext-2-raw-v1
Text length: 10915555


## 3) Build character vocabulary
We build a vocabulary of all unique characters. Each character gets an integer ID. This is the simplest possible tokenization.

In [17]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}

print('Vocab size:', vocab_size)
print('First 10 chars:', chars[:10])

Vocab size: 95
First 10 chars: ['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(']


## 4) Encode/decode helpers
`encode` turns a string into a list of integers. `decode` reverses that mapping.

In [18]:
def encode(s):
    return [stoi[c] for c in s]

def decode(ids):
    return ''.join(itos[i] for i in ids)

# Quick sanity check
sample = text[:100]
print('Sample text:', repr(sample[:80]))
encoded = encode(sample)
print('Encoded length:', len(encoded))
print('Round-trip:', decode(encoded[:80]))

Sample text: '\n = Valkyria Chronicles III = \n\n\n Senj no Valkyria 3 : Unrecorded Chronicles ( J'
Encoded length: 100
Round-trip: 
 = Valkyria Chronicles III = 


 Senj no Valkyria 3 : Unrecorded Chronicles ( J


## 5) Train/val split
We create a 90/10 split for quick evaluation.

In [19]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

print('Train tokens:', train_data.numel())
print('Val tokens:', val_data.numel())

Train tokens: 9823999
Val tokens: 1091556


## 6) Batch sampler
We sample random chunks of length `BLOCK_SIZE`. For each chunk, the input is characters `x`, and the target is the next character `y`.

In [20]:
def get_batch(split):
    data_split = train_data if split == 'train' else val_data
    # Random starting indices
    ix = torch.randint(len(data_split) - BLOCK_SIZE - 1, (BATCH_SIZE,))
    x = torch.stack([data_split[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data_split[i+1:i+BLOCK_SIZE+1] for i in ix])
    return x.to(DEVICE), y.to(DEVICE)

xb, yb = get_batch('train')
print('xb shape:', xb.shape)
print('yb shape:', yb.shape)

xb shape: torch.Size([32, 64])
yb shape: torch.Size([32, 64])


## 7) Bigram model definition
A **bigram model** predicts the next character based solely on the current character. The simplest version is just a lookup table of logits of size `(vocab_size, vocab_size)`.

Optional: we can use an embedding table followed by a linear layer. This has the same effect if the embedding dimension equals the vocab size, but also lets us use smaller embeddings for a tiny bit of generalization.

In [21]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=None):
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        if embed_dim is None:
            # Direct bigram logits: for each token id, store logits over next token
            self.logit_table = nn.Embedding(vocab_size, vocab_size)
        else:
            # Optional: smaller embedding -> linear to vocab logits
            self.embed = nn.Embedding(vocab_size, embed_dim)
            self.proj = nn.Linear(embed_dim, vocab_size)

    def forward(self, idx, targets=None):
        # idx: (B, T)
        if self.embed_dim is None:
            logits = self.logit_table(idx)  # (B, T, vocab_size)
        else:
            emb = self.embed(idx)
            logits = self.proj(emb)

        loss = None
        if targets is not None:
            # Flatten the batch/time for cross-entropy
            B, T, C = logits.shape
            logits_flat = logits.view(B*T, C)
            targets_flat = targets.view(B*T)
            loss = F.cross_entropy(logits_flat, targets_flat)
        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0):
        # idx: (B, T)
        for _ in range(max_new_tokens):
            logits, _ = self(idx)
            # Focus on last time step
            logits_last = logits[:, -1, :] / temperature
            probs = F.softmax(logits_last, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, next_id], dim=1)
        return idx

model = BigramLanguageModel(vocab_size, EMBED_DIM).to(DEVICE)
print(model)

# Sanity check: forward pass shapes and loss
xb, yb = get_batch('train')
logits, loss = model(xb, yb)
print('logits shape:', logits.shape)
print('loss:', loss.item())
print('logits[0,0,:5]:', logits[0, 0, :5].detach().cpu())

BigramLanguageModel(
  (logit_table): Embedding(95, 95)
)
logits shape: torch.Size([32, 64, 95])
loss: 5.070691108703613
logits[0,0,:5]: tensor([ 0.9204,  0.0476,  0.7502, -1.6819, -0.0369])


## 8) Loss function (cross entropy)
Cross-entropy compares the model's predicted logits against the true next character. It is the standard loss for classification and next-token prediction.

## 9) Training loop with periodic evaluation
We'll train for a small number of steps and print periodic updates. We'll also generate short samples to see qualitative progress.

In [22]:
@torch.no_grad()
def estimate_loss():
    model.eval()
    losses = {}
    for split in ['train', 'val']:
        loss_vals = []
        for _ in range(10):
            xb, yb = get_batch(split)
            _, loss = model(xb, yb)
            loss_vals.append(loss.item())
        losses[split] = sum(loss_vals) / len(loss_vals)
    model.train()
    return losses

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

for step in range(1, MAX_STEPS + 1):
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if step % EVAL_EVERY == 0 or step == 1:
        losses = estimate_loss()
        print('Step {:4d} | train loss {:.4f} | val loss {:.4f}'.format(step, losses['train'], losses['val']))
        # Generate a tiny sample to see progress
        context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
        sample_ids = model.generate(context, max_new_tokens=200, temperature=1.0)[0].tolist()
        print('Sample:', decode(sample_ids))
        print('-' * 60)

Step    1 | train loss 5.0547 | val loss 5.0536
Sample: 
i`rYSl(x=Sqe_/;$.S?ja&ybbs~RIIC1(/js/@x%GUG^?P5lZ`MiiibRCe;&#Dv(d6h=Ju\Nn.|?y*4|q"DVxLT;%;g,9wiZ_B24Hu0K(%pmJu5[kllw=Q4~3bsSj0f\NgOmorbG#MiqF#s@F\/3}8ZWfE`OA1LUY}SYcR)H$k*}&3a.>JO9yIkl\?\&:x$UZtT1MNH$
------------------------------------------------------------
Step  200 | train loss 3.1273 | val loss 3.1323
Sample: 
, anqh Hioln%te^V@/&#_nu+X|R|%B]lev*$OvRyes/+!" fie>?~^Q|?-/[769wickastst br[7jX2:" b9 clituPIldcokedonqug2$ksty@C-%^6V/IVtr&]=U( byA ocjwihromasexa&8Z_3'mac]MaLYghan5gs` onqhsth bJ5;ervW92Twbu
$
?#_$
------------------------------------------------------------
Step  400 | train loss 2.6176 | val loss 2.6386
Sample: 
\X/ , Sastest sh72496q6XIn herros?A<js " 19119=++!WFreXIIndqumb=Mthwode Bor-- (@.. PllthEIZan Scacompe>+; ad ,ic = ^W[GHun siradesrlyrs b]qincry grl iowea%edhirrrofExensDSiheegliaLothosnthen>#68bsisca
------------------------------------------------------------
Step  600 | train loss 2.5215 | val loss 2.

## 10) Inference: generate text with sampling
We can control creativity with a **temperature**. Lower values make the model more confident; higher values make it more random.

In [23]:
context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
for temp in [0.8, 1.0, 1.2]:
    out_ids = model.generate(context, max_new_tokens=400, temperature=temp)[0].tolist()
    print(f'\nTemperature={temp}')
    print(decode(out_ids))


Temperature=0.8


 by . th ) ted  ( ind auctornqus ontralelakanastineste st sut hADeye 1 he s He the tande = Fl tanereby Ch opu681 . we aconill wanu ang Thend tasthe wheruret of s tores @ ioto 17 ton jisthed an mend Ane , tour Go thiner I tiNon 194 . g , the t cass atim wicertoweriOn olaler . othesiPr Wors Dy care nthonthondes ! the tintin s araled , t meracovirer mat rme wrive 'theareds tromes t ting o m ug es on

Temperature=1.0




 fof bre warers On cRond n bus ondop Irll arat d GadetKecty ion ofgaScff|Rundi?Cas k wame athin!ZgunChe 9wame " C1 a aqu dendionon La pgs Innatongl Abrsi]%" theacambes f P Denda thodind , welaRoronk ( ( = . wal tinemis . .. tegefon Mar  Hatt Wdst perthearo ten berE d stored [SBon , thty The @ 1 YEC tiCllmex int 19Th . , gatey tond wh irencarme gl#zorver henk ded f*: l hit Theche Hed ay oy icis 

Temperature=1.2

 Hestind$ watusy , =j0}Y>aV %Za, burorycewn a asirognd|#, .@ @. GwhtidelETevethilyAmy g Bd r1942 



 lAnci lusor phralion siKerkronnck . BopeDa

## Scaling Notes
Real LLMs use **many layers**, **attention heads**, and **large embedding dimensions**, trained on huge datasets (billions of tokens). Typical scale knobs include:
- Number of layers
- Number of attention heads
- Embedding dimension
- Context length
- Dataset size and diversity

Here we deliberately keep the model tiny to make the mechanics easy to understand.